In [1]:
import warnings
warnings.filterwarnings("ignore")

from nilearn import datasets # Fetch data using nilearn
adhd = datasets.fetch_adhd(n_subjects=10)          # ADHD200 preprocessed data (Athena pipeline)

In [2]:
adhd.func[0]

'/home/jovyan/nilearn_data/adhd/data/0010042/0010042_rest_tshift_RPI_voreg_mni.nii.gz'

In [3]:
# Load voxel-level data - a brain mask is automatically computed
from nilearn import plotting, input_data
masker_voxel = input_data.NiftiMasker(detrend=True, standardize=True, smoothing_fwhm=5).fit(adhd.func)
tseries_voxel = []
for mm in range(10):
    tseries_voxel.append(masker_voxel.transform(adhd.func[mm]))

In [4]:
for mm in range(10):
    print(f"Time series with shape {tseries_voxel[mm].shape} (# time points, # voxels))")

Time series with shape (176, 85878) (# time points, # voxels))
Time series with shape (176, 85878) (# time points, # voxels))
Time series with shape (176, 85878) (# time points, # voxels))
Time series with shape (175, 85878) (# time points, # voxels))
Time series with shape (77, 85878) (# time points, # voxels))
Time series with shape (77, 85878) (# time points, # voxels))
Time series with shape (261, 85878) (# time points, # voxels))
Time series with shape (260, 85878) (# time points, # voxels))
Time series with shape (260, 85878) (# time points, # voxels))
Time series with shape (261, 85878) (# time points, # voxels))


In [75]:
_, N = tseries_voxel[0].shape
K = 20
Y = tseries_voxel
import numpy as np
from scipy.special import gamma

JE = 0;

beta_ = np.ones(K)
eta_ = np.ones(K)
lambda_ = 0.5*np.ones(K)
nu_ = (2*eta_ + len(Y) - 2)/(2*beta_)
a_ = (lambda_**(-1/beta_) * gamma(nu_ + 1/beta_)) / (len(Y) * gamma(nu_))

for kk in range(K):
    y_sub = np.zeros((len(Y),N))
    tot = 0
    for mm in range(len(Y)):
        ix = tot + 1
        y_sub[ix-1,:] = Y[mm][kk,:]
        tot = tot + 1

    D, t1 = np.linalg.eig(y_sub @ y_sub.T)
    A = ((t1 * (1 / D[None,:])) @ t1.T) @ y_sub
    z_k = np.sum(y_sub * A, axis=0)
    z_k_beta = z_k**beta_[kk]

    JE = JE + (lambda_[kk] * (((N-1)*a_[kk])**beta_[kk])/N) * np.sum(z_k_beta)


In [71]:
JE=0

In [76]:
JE + (lambda_[kk] * (((N-1)*a_[kk])**beta_[kk])/N) * np.sum(z_k_beta)

104.99877733528967